In [ ]:
principalId=""
principalSecret=""
dwName=""
dwDescription="New service principal owned data warehouse"
wksId=""

In [ ]:
def retrieveAuthHeader(clientId, clientSecret):
    import json, requests, pandas as pd 
    import datetime  
    tenant="52b6b910-1fc7-44e7-b03d-ffb4ea2dd90b"
    try: 
        from azure.identity import ClientSecretCredential 
    except Exception:
        !pip install azure.identity 
        from azure.identity import ClientSecretCredential 

    # Generates the access token for the Service Principal 
    api = 'https://analysis.windows.net/powerbi/api/.default' 
    auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/', 
                tenant_id = tenant, 
                client_id = clientId, 
                client_secret = clientSecret) 
    access_token = auth.get_token(api)
    access_token = access_token.token 
    ## This is where I store my header with the Access Token, because this is required when authenticating 
    ## to the Power BI Admin APIs 
    auth = {'Authorization': f'Bearer {access_token}'}  
    return auth

In [ ]:
def createDW(authHeader, payload,workspaceid):
    #shortcuts need to be excluded from the lakehouse maintenance
    import sempy.fabric as fabric
    from sempy.fabric.exceptions import FabricHTTPException
    import requests

    client=fabric.FabricRestClient()

    url=f"v1/workspaces/{workspaceid}/items"
    try:
        result=client.post(url,headers=auth,json=payload)
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        print(f"Response content: {result.text}")  # Print the response body
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")

In [ ]:
payload = { 
  "type": "warehouse", 
  "displayName": dwName, 
  "description": dwDescription, 
  "creationPayload": { 
    "defaultCollation": "Latin1_General_100_CI_AS_KS_WS_SC_UTF8" 
  } 
}

In [ ]:
auth=retrieveAuthHeader(principalId,principalSecret)

createDW(auth,payload,wksId)